Maybe it's just me, but I find the BERT code released by Google non-intuitive and difficult to use. It's a shame they didn't provide a couple of notebooks.

Fortunately, ktrain is a lightweight wrapper for Keras (similar to the fast.ai library, but for Keras), and it supports all kinds of pre-trained models, including BERT. Very cool project? Why isn't it more popular? More info at https://github.com/amaiya/ktrain/

This example shows how to fine-tune a pre-trained BERT model on a movie review dataset from IMDB.

In [2]:
!pip install -qU ktrain

You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import ktrain
from ktrain import text

Using TensorFlow backend.


using Keras version: 2.2.4


In [6]:
%%sh
wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
tar xvfz aclImdb_v1.tar.gz

--2019-10-08 15:29:41--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  10.3MB/s    in 10s     

2019-10-08 15:29:52 (8.01 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [9]:
%%sh
head aclImdb/train/pos/10245_10.txt

this was a personal favorite of mine when i was young, it had everything that was great with 90's kids movies... lovable dinosaurs, cute kids, an eccentric villain, and a few great songs (and not the typical little mermaid/beauty and the beast type songs, but ones that are atually entertaining)! i ran into this movie again recently and i still love it as much as ever! i recommend that everyone of every age should see this movie, and i definitely think that it should be introduced to the younger generations! sorry not the most informative, i'm in kinda a rush... just please, trust me. all who go against this movie are killing their inner child!

In [8]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder('aclImdb', 
                                                                       maxlen=500, 
                                                                       preprocess_mode='bert',
                                                                       train_test_names=['train', 
                                                                                         'test'],
                                                                       classes=['pos', 'neg'])

detected encoding: utf-8
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


preprocessing test...
language: en


In [10]:
model = text.text_classifier('bert', (x_train, y_train) , preproc=preproc)

Is Multi-Label? False
maxlen is 500
done.


In [11]:
learner = ktrain.get_learner(model, 
                             train_data=(x_train, y_train), 
                             val_data=(x_test, y_test), 
                             batch_size=6)

In [13]:
# 2e-5 is one of the LRs  recommended by Google and is consistent with the plot above.
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
Train on 25000 samples, validate on 25000 samples
Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor with shape[768,3072] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu
	 [[{{node training/Adam/mul_878}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
data = [ 'This movie was horrible! The plot was boring. Acting was okay, though.',
         'The film really sucked. I want my money back.',
        'The plot had too many holes.',
        'What a beautiful romantic comedy. 10/10 would see again!',
         ]

In [ ]:
predictor.predict(data)